## Read data from train.txt and filter it from unwanted patterns


In [1]:
from Preprocessing import utils

## Preprocessing


In [2]:
training_set = utils.read_data(f"./Dataset/train.txt")
print("training_set", training_set[0:200])
filtered_training_set = utils.filter_data(training_set)
print("filtered_training_set", filtered_training_set[0:200])


training_set قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ( 14 / 123 )
ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُ
filtered_training_set قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُ


In [4]:
chracter_set = utils.char_tokenizer(filtered_training_set)
print("chracter_set", chracter_set[0:10])

chracter_set ['أ', 'ق', 'ه', 'ْ', 'َ', 'ض', '،', 'ؤ', "'", '*']


## Feature Extraction


## Building The Model


## Model Evaluation
